In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import re

In [2]:
faq = pd.read_csv('./FAQ_List.csv', low_memory=False)
#faq = pd.read_csv('./공지2.csv', low_memory=False)

In [3]:
faq

,Unnamed: 0,FAQ,Answer
0,1,"[회원 가입 및 로그인]\n로그인 계정(아이디(ID), 패스워드(Password))...",홈>로그인>ID 및 PW 찾기 를 이용해주세요.
1,2,[회원 가입 및 로그인]\n회원 탈퇴를 하고 싶어요,"로그인 후, 홈>My Page>회원 탈퇴 메뉴를 이용해 주세요."
2,3,"[회원 가입 및 로그인]\n회원 탈퇴 후, 재가입을 하고 싶어요","회원 탈퇴를 하신후 재가입을 원하신다면, 탈퇴한 ID는 사용 불가합니다.\n기존과 ..."
3,4,[주문 / 결제 / 배송]\n주문 후 배송지 변경을 하고 싶어요,"배송지 변경은 My Page>주문배송조회>주문상세 에서 변경 가능하며, 자세한 내용..."
4,5,[주문 / 결제 / 배송]\n비회원도 주문이 가능한가요?,"네, 비회원으로도 주문 가능합니다.\n다만 비회원 주문시 쿠폰이용, 일부 상품 할인..."
5,6,"[취소 / 교환 / 환불]\n주문 취소시, 쿠폰도 환불되나요?","결제 완료 상태인 고객님의 주문 취소시, 적용된 쿠폰은 모두 복원 됩니다.\n하지만..."
6,7,[취소 / 교환 / 환불]\n카드 환불은 언제 되나요?,고객님의 카드 환불은 카드사의 확인이 영업일 기준 3~5일 정도 소요됩니다.
7,8,[취소 / 교환 / 환불]\n주문을 취소하고 싶어요,주문 취소는 '결제 완료' 단계의 상품만 가능합니다.\n이미 상품이 '배송중'이거나...
8,9,[취소 / 교환 / 환불]\n인터넷으로 주문한 상품은 매장에서 교환 및 환불이 되나요?,"온라인에서 주문한 상품은 출고된 매장에서만 교환 및 환불이 가능합니다.\n예를들어,..."
9,10,[상품]\n상품 및 서비스에 대한 문의는 어디서 해야하나요?,홈페이지내 고객센터 페이지에서 1:1 상담 또는 고객센터 대표 전화(02-xxxx-...


In [5]:
faq.Answer[0]

'홈>로그인>ID 및 PW 찾기 를 이용해주세요.'

In [6]:
# 코사인 유사도 

def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

In [7]:
from konlpy.tag import Okt
okt = Okt()
stopwords = ['은','는','이','가']

In [8]:
from pykospacing import spacing # 띄어쓰기 교정
from hanspell import spell_checker # 맞춤법 교정

In [10]:
temp = pd.DataFrame(columns=['FAQ'])
temp['FAQ'] = faq['FAQ'].copy()
temp['FAQ'] = temp['FAQ'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-zA-Z]"," ")

for i in range(len(temp)):   
    temp['FAQ'][i] = temp['FAQ'][i].lower()
    temp['FAQ'][i] = spacing(temp['FAQ'][i]) #띄어쓰기 교정
    temp['FAQ'][i] = spell_checker.check(temp['FAQ'][i]).checked # 맞춤법 교정
    
    temp['FAQ'][i] = okt.morphs(temp['FAQ'][i], stem=True)
    temp['FAQ'][i] = [word for word in temp['FAQ'][i] if not word in stopwords]
faq['FAQ2'] = temp['FAQ'].copy()

In [27]:
# 챗봇 사용자 채팅

usr_question = input("사용자 질문 : ")

사용자 질문 :  주문 취소


In [28]:
### 사용자 질문 처리

question = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-zA-Z]", " ", usr_question)
question = question.lower()
question = spacing(question)
question = spell_checker.check(question).checked
question = okt.morphs(question, stem=True)
question = [word for word in question if not word in stopwords]

print(question)   

['주문', '취소']


In [30]:
question = ' '.join(question)
temp_list = [question]

for i in range(len(faq['FAQ2'])):
    join_str = ' '.join(faq['FAQ2'][i])
    temp_list.append(join_str)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(temp_list)
#print(feature_vect_simple.shape)
#print(type(feature_vect_simple))

# TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환.
feature_vect_dense = feature_vect_simple.todense()

#첫번째 문장과 두번째 문장의 feature vector  추출
#vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
#vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

#첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
#similarity_simple = cos_similarity(vect1, vect2)
#print('문장 1, 문장 2 Cosine 유사도: {0:.3f}'.format(similarity_simple))

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

#similarity_simple_pair = cosine_similarity(feature_vect_simple[0] , feature_vect_simple)
#print(similarity_simple_pair)

similarity_simple_pair = cosine_similarity(feature_vect_simple , feature_vect_simple)
#print(similarity_simple_pair)
#print(similarity_simple_pair[0][1:])
result = similarity_simple_pair[0][1:]
print(result)

[0.         0.         0.         0.31917922 0.34896902 0.53945958
 0.19852764 0.62794778 0.27042416 0.         0.        ]


In [33]:
val_index = np.argmax(result)
result_dic = dict(enumerate(result))
res = sorted(result_dic.items(), key=(lambda x: x[1]), reverse = True)

In [34]:
print("질문 : " + usr_question)
#print("질문(필터링) : " + question + "\n")
print("\n[유사도 기반 FAQ List Top3]\n")

sum = 0
seq = 1
for k, v in res[:3]:
    
    if v != 0:
        print(seq, ".", faq['FAQ'][k])
        print(faq['Answer'][k])
        print("\n")
        sum = sum+1
        seq = seq+1
        

if sum == 0:
    print("검색결과가 없습니다.")
    print("필수 단어 위주로 검색해보세요.")
    
else:
    #print('\n')
    print("※찾고자하는 답변이 아니신가요? 그렇다면 필수 단어 위주로 검색해보세요.")

질문 : 주문 취소

[유사도 기반 FAQ List Top3]

1 . [취소 / 교환 / 환불]
주문을 취소하고 싶어요
주문 취소는 '결제 완료' 단계의 상품만 가능합니다.
이미 상품이 '배송중'이거나 '배송 완료'의 경우에는 수령 후 교환 및 반품신청을 해주시기 바랍니다.


2 . [취소 / 교환 / 환불]
주문 취소시, 쿠폰도 환불되나요?
결제 완료 상태인 고객님의 주문 취소시, 적용된 쿠폰은 모두 복원 됩니다.
하지만 사용기한이 지난 쿠폰의 경우에는 쿠폰 재발행은 불가합니다.


3 . [주문 / 결제 / 배송]
비회원도 주문이 가능한가요?
네, 비회원으로도 주문 가능합니다.
다만 비회원 주문시 쿠폰이용, 일부 상품 할인, 행사 적용, 이벤트 참여 등의 회원 혜택이 제한됩니다.


※찾고자하는 답변이 아니신가요? 그렇다면 필수 단어 위주로 검색해보세요.
